<a href="https://colab.research.google.com/github/WZRP/mass-watermarker/blob/main/Mass-Watermarker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

⚠ Please open Colab Files Explorer to Upload the Watermark Image yourself.

In [ ]:
#@title # ⬇ Download and extract ZIP or RAR file
import os
import io
import requests

# Define the URL of the file to download
url = "" #@param {'type': 'string'}

# Define the name of the local directory to extract to
extract_dir = "" #@param {'type': 'string'}

# Create the directory if it doesn't exist
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

# Create the directory if it doesn't exist
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

# Download the file and save it to memory
response = requests.get(url)
data = response.content

# Extract the contents of the file to the directory based on the file extension
if url.endswith(".zip"):
    import zipfile
    with zipfile.ZipFile(io.BytesIO(data)) as zip_ref:
        zip_ref.extractall(extract_dir)
elif url.endswith(".rar"):
    !pip install rarfile
    import rarfile
    with rarfile.RarFile(io.BytesIO(data)) as rar_ref:
        rar_ref.extractall(extract_dir)
else:
    print("Unsupported file type. Please provide a ZIP or RAR file.")

In [ ]:
#@title # 🔀 Convert all Images to PNG
import os
from PIL import Image

# Define the directory path
dir_path = "" #@param {'type': 'string'}

# Loop through all files in the directory
for filename in os.listdir(dir_path):
    # Check if the file is an image (JPG or JPEG)
    if filename.lower().endswith(('.jpg', '.jpeg')):
        # Rename the file to have a .png extension
        os.rename(os.path.join(dir_path, filename), os.path.join(dir_path, os.path.splitext(filename)[0] + ".png"))

In [ ]:
#@title # 📝 Mass Rename Images to 0000 -> 9999
import os

# Define the directory path
folder_dir = ""  #@param {'type': 'string'}

# Define the starting number for the renaming
start_num = 0

# Loop through all files in the directory
for filename in os.listdir(folder_dir):
    # Check if the file is an image (JPG, JPEG, or PNG)
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        # Generate the new filename with a four-digit number
        new_filename = f"{start_num:04}.png"
        # Rename the file with the new filename
        os.rename(os.path.join(folder_dir, filename), os.path.join(folder_dir, new_filename))
        # Increment the start number for the next file
        start_num += 1

In [ ]:
#@title # Image Dimension Viewer
import os
from PIL import Image

# Define the input directory path
input_dir = "/content/raw"

# Create an empty list to store the dimensions of all images
dimensions = []

# Loop through all files in the input directory
for filename in os.listdir(input_dir):
    # Check if the file is an image (JPEG or PNG)
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        # Open the image file
        input_filepath = os.path.join(input_dir, filename)
        with Image.open(input_filepath) as img:
            # Get the dimensions of the image
            width, height = img.size
            # Add the filename and dimensions to the list
            dimensions.append((filename, width, height))

# Print the list of filenames and dimensions
for filename, width, height in dimensions:
    print(f"{filename}: {width}x{height}")

In [ ]:
#@title # ™ Mass Insert Watermark
import os
import tqdm
from PIL import Image

# Watermark image file
watermark_file = '' #@param {'type': 'string'}

# Folder of images to watermark
images_folder = '' #@param {'type': 'string'}

# Output folder for watermarked images
output_folder = '' #@param {'type': 'string'}


# Create output folder if doesn't exist
if not os.path.exists(output_folder):
  os.makedirs(output_folder)

# Load watermark image
watermark = Image.open(watermark_file)

# Get number of images to process
num_images = len(os.listdir(images_folder))

# Sort image names alphabetically
image_files = sorted(os.listdir(images_folder))

# Process images with progress bar
with tqdm.tqdm(total=num_images) as pbar:
  for i, image_file in enumerate(image_files):

    # Open image
    try:
      image = Image.open(os.path.join(images_folder, image_file))
    except OSError as e:
      print(f"Error opening {image_file}: {e}")
      continue

    # Watermark image
    w, h = watermark.size
    padding = 128
    image.paste(watermark, (padding, image.height - h - padding), watermark)

    # Save watermarked image as PNG
    output_path = os.path.join(output_folder, os.path.splitext(image_file)[0] + '.png')
    image.save(output_path)

    # Update progress bar
    pbar.update(1)

print('\nWatermarked images saved to', output_folder)

In [ ]:
#@title # ✔ Mass Insert Watermark (Entire Folder Directory)
!pip install Pillow
import os
import tqdm
from PIL import Image

mother_folder_var = "" #@param {'type': 'string'}
watermark_var = "" #@param {'type': 'string'}

#@markdown &nbsp; <font color=gray><i> The code inserts a watermark image into all image files in a specified directory and its subdirectories. It finds all image files in the directory tree and creates a new folder called "Logo - Brand" in each directory containing image files. The watermarked images are saved in this folder with the same format as the original images.<br>

# Set the paths to the mother folder and watermark image
mother_folder_path = os.path.abspath(mother_folder_var)
watermark_path = os.path.abspath(watermark_var)

# Define a function to process each image file
def process_image(image_file):
    # Create a new Logo - Brand folder in the current directory
    new_folder_path = os.path.join(os.path.dirname(image_file), "Logo - Brand")
    os.makedirs(new_folder_path, exist_ok=True)
    # Create a new copy of the original image in the Logo - Brand folder
    new_image_path = os.path.join(new_folder_path, os.path.basename(image_file))
    with Image.open(image_file) as image:
        # Open the watermark image
        with Image.open(watermark_path) as watermark:
            # Calculate the position of the watermark image
            watermark_position = (128, image.height - watermark.height - 128)
            # Paste the watermark image onto the image
            image.paste(watermark, watermark_position, mask=watermark)
            # Save the modified image with the same format as the original
            output_path = new_image_path
            image.save(output_path, format=image.format)

# Process all image files in the directory tree
for root, dirs, files in os.walk(mother_folder_path):
    for file in tqdm.tqdm(files):
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(root, file)
            process_image(image_path)

print('\nWatermarking complete.')

In [ ]:
#@title # ZIP Folder
import os
import zipfile
import tqdm

# Define the input directory path, the desired name of the ZIP file,
# and the desired output directory for the ZIP file
input_dir = "" #@param {'type': 'string'}
zip_filename = "" #@param {'type': 'string'}
output_dir = "" #@param {'type': 'string'}

# Create a new ZIP file
output_zip = os.path.join(output_dir, zip_filename)
with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zip_file:
    # Get the number of files to add to the ZIP archive
    num_files = sum(len(files) for _, _, files in os.walk(input_dir))

    # Loop through all files in the input directory and add them to the ZIP archive
    with tqdm.tqdm(total=num_files) as pbar:
        for root, dirs, files in os.walk(input_dir):
            for file in files:
                # Get the full path of the file
                file_path = os.path.join(root, file)
                # Add the file to the ZIP archive
                zip_file.write(file_path, arcname=os.path.relpath(file_path, input_dir))
                # Update the progress bar
                pbar.update(1)

print(f"\nZIP archive created successfully at {output_zip}")

In [ ]:
#@title # Login to Huggingface
!pip install huggingface_hub
try:
  hub_ok # My packaged dep didn't contains this (but stil have the folder)
except:
  print("Setting up huggingface_hub...")
  !pip install --force-reinstall -qqqq huggingface_hub
  hub_ok = True
from IPython.display import clear_output
from huggingface_hub import login
clear_output()

#@markdown # <b> [🤗 Click here to get your Huggingface token](https://huggingface.co/settings/tokens)

write_token = "" #@param {type:"string"}
login(write_token, add_to_git_credential=True)

In [ ]:
#@title # Setup Repo
from huggingface_hub.utils import validate_repo_id, HfHubHTTPError
from huggingface_hub import HfApi


api = HfApi()
user = api.whoami(write_token)

#@markdown #### If your model repo didn't exist, it will automatically create your repo.
repo_name = "" #@param{type:"string"}
make_this_repo_private_if_not_exist = False #@param{type:"boolean"}
clone_with_git = False #@param{type:"boolean"}

model_repo = user['name']+"/"+repo_name.strip()

validate_repo_id(model_repo)

if repo_name != "":
  try:
      api.create_repo(repo_id=model_repo,
                      private=make_this_repo_private_if_not_exist)
      print("Model Repo didn't exists, creating repo")
      print("Model Repo: ",model_repo,"created!\n")

  except HfHubHTTPError as e:
      print(f"Model Repo: {model_repo} exists, skipping create repo\n")

if clone_with_git:
  !git lfs install --skip-smudge
  !export GIT_LFS_SKIP_SMUDGE=1
  !git clone https://huggingface.co/{model_repo} /content/{repo_name}

In [ ]:
#@title # Upload to Huggingface Repo
from huggingface_hub import HfApi
from pathlib import Path

api = HfApi()
file_path = "" #@param {type :"string"}
commit_message = "" #@param {type :"string"}

if file_path != "":
  path_obj = Path(file_path)
  trained_model = path_obj.parts[-1]

  print(f"Uploading {trained_model} to https://huggingface.co/"+model_repo)
  print(f"Please wait...")

  api.upload_file(
      path_or_fileobj=file_path,
      path_in_repo=trained_model,
      repo_id=model_repo,
      commit_message=commit_message,
  )

  print(f"Upload success, located at https://huggingface.co/"+model_repo+"/blob/main/"+trained_model+"\n")